# HOMA-IR - Massive DL Ablation Study


 - Input Features
 - Time Windows
 - Cross Validation Method
 - Masked Autoencoders

# Library requirements

The libraries used in this colab are standard for [colab.research.google.com](https://colab.research.google.com)

In case of import failures, manually install the failing packages
by running `!pip install <package>` in a new cell.


# Run required imports

In [1]:
import numpy as np

import os

import pandas as pd
import pickle

import shap

from sklearn import model_selection, metrics
from sklearn.metrics import precision_score, recall_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import time

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from typing import Literal, Dict, List

import xgboost

In [2]:
# Colab specifics
from google.colab import files
from google.colab import sheets

import warnings
warnings.filterwarnings('ignore')

# Define paths & upload data

File uploads and algorithm outputs can be explored under the "Files" tab in the menu on the left.

In [3]:
# Define paths
INPUTS_FOLDER = '/input_data'
OUTPUTS_FOLDER = '/output_data'

EXPERIMENT_NAME = 'autoencoder_ablation_study_results'
OUTPUTS_K_FOLD_ABLATION_FOLDER = os.path.join(OUTPUTS_FOLDER, EXPERIMENT_NAME)
OUTPUTS_MODELS_PATH = os.path.join(OUTPUTS_K_FOLD_ABLATION_FOLDER, "ablation_models")

In [4]:
# Create paths
path_list = [INPUTS_FOLDER, OUTPUTS_FOLDER, OUTPUTS_K_FOLD_ABLATION_FOLDER, OUTPUTS_MODELS_PATH]

for path in path_list:
  if not os.path.exists(path):
    os.makedirs(path)


In [5]:
# @title Upload file from your filesystem.
uploaded = files.upload(target_dir=INPUTS_FOLDER)

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes to folder {folder}'.format(
      name=fn, length=len(uploaded[fn]), folder=INPUTS_FOLDER))

if len(uploaded) == 1:
  DATA_SOURCE_PATH = fn
  print('Using file: ', DATA_SOURCE_PATH)
else:
  print('Only one file supported, try again.')

Saving data.pq to /input_data/data.pq
User uploaded file "/input_data/data.pq" with length 423429 bytes to folder /input_data
Using file:  /input_data/data.pq


In [11]:
DATA_SOURCE_PATH

'input_data/data.pq'

In [12]:
# @title Session storage upload
# Load file from a given path. Paths are local to the session storage.
DATA_SOURCE_PATH = '/input_data/data.pq'  # @param {type:"string"}
print('Using file: ', DATA_SOURCE_PATH)

Using file:  /input_data/data.pq


In [13]:
# @title Load data
with open(DATA_SOURCE_PATH, 'rb') as infile:
  training_data_aggregated_all_days = pd.read_parquet(infile)

(1173, 154)

# Helper classes and functions

### Helper for storing and benchmarking traditional ML models

In [16]:
# Constants
RANDOM_SEED = 2024

XGBOOST_DEFAULT_PARAM_GRID = {
    "n_estimators": [50, 100, 200],
    "learning_rate": [0.01, 0.1, 0.3],
    "max_depth": [3, 5, 7],
    "reg_lambda": [0.1, 1, 10],
    "reg_alpha": [0.1, 1, 10],
    "booster": ["gbtree", "gblinear"],
    "random_state": [RANDOM_SEED],
}
RANDOM_FOREST_DEFAULT_PARAM_GRID = {
    "n_estimators": [100, 200],
    "max_depth": [None, 5, 10],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "max_features": ["auto", "sqrt"],
    "bootstrap": [True, False],
    "random_state": [RANDOM_SEED],
}
SVM_DEFAULT_PARAM_GRID = {
    "C": [0.1, 1, 10],
    "kernel": ["linear", "rbf", "poly"],
    "gamma": ["scale", "auto", 0.1, 1],
    "random_state": [RANDOM_SEED],
}
KNN_DEFAULT_PARAM_GRID = {
    "n_neighbors": [2, 3, 5, 7, 10],
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "cosine"],
}
LOGISTIC_REG_DEFAULT_PARAM_GRID = {
    "penalty": ["l1", "l2"],
    "C": [0.01, 0.1, 1, 10],
    "solver": ["liblinear", "saga"],
}
DECISION_TREE_DEFAULT_PARAM_GRID = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 4, 6, 8],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5],
    "random_state": [RANDOM_SEED],
}

In [17]:
class MLModelBenchmarking:
  """A class for benchmarking and storing traditional ML models.

  This class aims to provide an end-to-end standardized tool for benchmarking
  ML models across different Metabolic Health tasks with minimal inputs and
  configurations from the user, though many parameters can be passed to
  various methods for more specific behaviors and experiments.

  Main Functionalities:
  - Train any of the supported ML models (only with specifying a string)
  - Run grid search on default grids, or user-provided grids
  - Run standard evaluation for classification and regression tasks based on
    our pre-determined performance goals
  - Save all required information for reproducibility and running the models
    in the future as a self-contained object.**
  - Export performance metrics, path to reproducibility object, and
    experimental settings to a shared gSheets for future benchmarking and
    reproducibility

  Attributes:
    data_points: A numpy array containing the datapoints used for training and
      evaluation.
    targets: A numpy array containing the targets/labels.
    participant_ids: A numpy array containing the subject/participant IDs.
    ml_model_name: A string indicating the name of the ML model that we need to
      train and evaluate. Currently, the string
    prediction_dataframe: A dataframe containing subject IDs in the test set,
      index of cross validation split, true labels, and predicted labels.
    verbose: A boolean indicating whether to print verbose output.
    predicted_participant_ids: A numpy array containing the predicted
      participant IDs.
    true_labels_through_cv: A numpy array containing the true labels.
    split_indices: A numpy array containing the split indices.
    predicted_labels_through_cv: A numpy array containing the predicted labels.
    model: The classification or regression model that we trained.
    model_evaluation_metrics: A dictionary mapping the name of desired
      evaluation metrics, such as 'classification report' or 'confusion matrix',
      to the computed values the values.
    task_code: An integer indicating the task to perform (1 for classification,
      2 for regression).
    grid_search_metric: A string indicating the grid search metric.
    user_provided_task: A string provided by the user indicating the task to
      perform. Currently, the string can be 'classification' or 'regression'.
    url_to_gsheets: A string indicating the URL to gSheets.
    neg_log_probability: A numpy array indicating the negative log probability.
    random_seed: An integer indicating the random seed.
    name_of_experiment: A string indicating the name of the experiment.
    label_integers_to_name_dictionary: A dictionary mapping labels (integers) to
      strings indicating the meaning of each label.
    number_of_synthetics_per_label: A integer indicating the number of
      synthetics per label.
    number_of_neighbors_to_use: A integer indicating the number of neighbors to
      use.
    cross_val_mode: A string indicating which type of cross validation we
      performed.
    augmentation_verbose: A boolean indicating whether to augment training data.
    model_parameters: A dictionary containing model parameters.
    parameter_grid: A dictionary containing grid search parameters.
    augment_training_data: A boolean indicating whether to augment training
      data.
    n_neighb_for_synth: The number of neighbors used for generating synthetic
      data.
    per_class_eval_dataframe: A pandas dataframe that contains classification
      metrics per class.
    confusion_matrix: The computed confusion matrix if the desired task is
      classification.
    classification_report: A string for visualizing results that stores computed
      classification metrics, if the desired task is classification.
    probability: A numpy array containing the probability of each class.
    prediction_results_dataframe: A pandas dataframe that contains the
      prediction results for each participant.
  """

  def __init__(
      self,
      data_points_and_labels_df: pd.DataFrame,
      targets_column_name: str = None,
      subject_id_column_name: str = None,
      task: Literal["classification", "regression"] = "classification",
      ml_model: Literal[
          "random_forest",
          "svm",
          "logistic_regression",
          "knn",
          "xgboost",
          "decision_tree",
      ] = "xgboost",
      name_of_experiment: str = None,
      label_integers_to_name_dictionary: Dict[int, str] = None,
      verbose: bool = False,
      random_seed: int = None,
  ) -> None:
    """Constructor for the class BenchmarkXGBoostModel.

    Args:
      data_points_and_labels_df: A dataframe containing the data points, targets
        or labels, and subject/participant IDs.
      targets_column_name: A string indicating the name of the column in the
        dataframe containing the targets/labels.
      subject_id_column_name: A string indicating the name of the column in the
        dataframe containing the subject/participant IDs.
      task: A string indicating the task to perform.
      ml_model: A string indicating the name of the ML model.
      name_of_experiment: A string indicating the name of the experiment.
      label_integers_to_name_dictionary: A dictionary mapping labels (integers)
        to strings indicating the meaning of each label.
      verbose: A boolean indicating whether to print verbose output.
      random_seed: An integer indicating the random seed.

    Raises:
      ValueError: If 'targets_column_name' is not provided (remains None).
      ValueError: If 'subject_id_column_name' is not provided (remains None).
      ValueError: If 'task' is not classification or regression.
    """
    # This value is stored in the object for future identifiability.
    self.name_of_experiment = name_of_experiment
    self.label_integers_to_name_dictionary = label_integers_to_name_dictionary

    if targets_column_name is None:
      raise ValueError(
          "Please indicate which column of the passed dataframe "
          "contains the targets/labels through setting variable "
          "'targets_column_name'."
      )

    if subject_id_column_name is None:
      raise ValueError(
          "Please indicate which column of the passed dataframe "
          "contains subject/participant IDs through setting "
          "variable 'subject_id_column_name'."
      )

    self.participant_ids = data_points_and_labels_df[
        subject_id_column_name
    ].values
    self.targets = data_points_and_labels_df[targets_column_name].values
    self.data_points = data_points_and_labels_df.drop(
        columns=[subject_id_column_name, targets_column_name]
    ).values
    if random_seed:
      self._random_seed = random_seed
    else:
      # This is a constant that needs to be imported from the constants file
      # once we turn the notebook into a library.
      self._random_seed = RANDOM_SEED

    self.verbose = verbose
    self.true_labels_through_cv = None
    self.predicted_labels_through_cv = None
    self.model_evaluation_metrics = {}
    self.ml_model_name = ml_model
    if task.lower() == "classification":
      self.task_code = 1
      self.grid_search_metric = None
    elif task.lower() == "regression":
      self.task_code = 2
      self.grid_search_metric = "r2"
    else:
      raise ValueError(
          f"Unknown task: {task}. Please make sure to provide"
          "either 'classification' or 'regression' as task."
      )
    self.user_provided_task = task.lower()
    self.url_to_gsheets = None
    self.neg_log_probability = None

  def run_task_pipeline(
      self,
      model_parameters: Dict[str, List[int | float]] = None,
      type_of_cross_validation: Literal["loocv", "kfold"] = "loocv",
      parameter_grid: Dict[str, List[int | float]] = None,
      augment_training_data: bool = False,
      number_of_synthetics_per_label: Dict[str, List[int | float]] = None,
      number_of_neighbors_to_use: int = 5,
      augmentation_verbose: bool = False,
  ) -> None:
    """Initiates benchmarking and performing the desired task.

    Args:
      model_parameters: a dictionary containing model parameters. If left as
        None, method will perform grid search with default parameters based on
        self.ml_model_name attribute set by the constructor.
      type_of_cross_validation: A string indicating the type of cross
        validation, either 'loocv' or 'kfold'.
      parameter_grid: A dictionary containing grid search parameters. If left as
        None, method will get default grid search based on ML model ( indicated
        by self.ml_model_name).
      augment_training_data: A boolean indicating whether to augment training
        data.
      number_of_synthetics_per_label: A dictionary containing the amount of
        synthetics samples we want to add to training data per label.
      number_of_neighbors_to_use: An integer indicating the number of neighbors
        to use.
      augmentation_verbose: A boolean indicating whether to print verbose
        output.

    Returns:
      Method results a dictionary mapping the name of desired evaluation
      metrics to the computed values the values.

    Raises:
      None.
    """
    start_time = time.time()
    self.parameter_grid = parameter_grid
    self.cross_val_mode = type_of_cross_validation
    self.augment_training_data = augment_training_data
    self.number_of_synthetics_per_label = number_of_synthetics_per_label
    if number_of_neighbors_to_use is not None:
      self.n_neighb_for_synth = number_of_neighbors_to_use
    else:
      self.n_neighb_for_synth = 5
    self.augmentation_verbose = augmentation_verbose

    if model_parameters is None:
      if self.verbose:
        print(
            "==> Model parameters were not provided.\n"
            "    -> Performing grid search."
        )
      self._run_grid_search()
    else:
      self.model_parameters = model_parameters

    if self.verbose:
      print(f"==> Performing downstream task with {self.cross_val_mode}:")
    self._run_downstream_task()

    if self.verbose:
      print(
          f"==> Computing {self.user_provided_task} metrics with "
          f"{self.cross_val_mode} cross validation:"
      )
    if self.task_code == 1:
      (
          self.model_evaluation_metrics["per_class_eval_dataframe"],
          self.model_evaluation_metrics["confusion_matrix"],
          self.model_evaluation_metrics["class_report"],
      ) = self.compute_classification_metrics(
          self.true_labels_through_cv,
          self.predicted_labels_through_cv,
          return_results=True,
      )
    else:
      self.model_evaluation_metrics["r2_score"] = (
          self.compute_regression_metrics(
              self.true_labels_through_cv, self.predicted_labels_through_cv
          )
      )
    if self.verbose:
      if self.task_code == 1:
        print(
            "==> Returning evaluation dictionary with confusion matrix and "
            "classification report."
        )
      else:
        print("==> Returning evaluation dictionary with r^2 score. ")
      print(
          f">-< Done. Running task pipeline took {time.time() - start_time} "
          "seconds."
      )

    return self.model_evaluation_metrics

  def compute_classification_metrics(
      self,
      true_labels: np.array,
      predicted_labels: np.array,
      return_results: bool = False,
  ) -> tuple[pd.DataFrame, np.ndarray, np.ndarray]:
    """Class method for computeing appropriate classification metrics.

    Args:
      true_labels: a numpy array containing true labels.
      predicted_labels: a numpy array containing predicted labels.
      return_results: A boolean indicating whether we want the results to be
        returned as well as storing them internally in the object.

    Returns:
      Class method will return the confusion matrix and the classification
      report.

    Raises:
      None.
    """
    res = []
    for current_class in np.unique(self.true_labels_through_cv):
      precision, recall, _, _ = metrics.precision_recall_fscore_support(
          self.true_labels_through_cv == current_class,
          self.predicted_labels_through_cv == current_class,
          pos_label=True,
          average=None,
      )
      try:
        res.append([current_class, recall[0], recall[1], precision[1]])
      except IndexError as e:
        print(
            "There seems to be an issue with the number of classes. "
            "E.g, there may be only one class. Please check labels and "
            " try again. Error: ",
            e,
        )
    self.per_class_eval_dataframe = pd.DataFrame(
        res, columns=["class", "specificity", "sensitivity", "precision"]
    )
    self.per_class_eval_dataframe["class"] = self.per_class_eval_dataframe[
        "class"
    ].astype(int)
    if self.label_integers_to_name_dictionary:
      self.per_class_eval_dataframe["class"] = self.per_class_eval_dataframe[
          "class"
      ].map(self.label_integers_to_name_dictionary)

    self.confusion_matrix = metrics.confusion_matrix(
        true_labels, predicted_labels
    )
    self.classification_report = metrics.classification_report(
        true_labels, predicted_labels
    )

    if return_results:
      return (
          self.per_class_eval_dataframe,
          self.confusion_matrix,
          self.classification_report,
      )

  def compute_regression_metrics(
      self, true_labels: np.array, predicted_labels: np.array
  ) -> np.ndarray:
    """Class method for computeing appropriate regression metrics.

    Args:
      true_labels: a numpy array containing true targets.
      predicted_labels: a numpy array containing predicted targets.

    Returns:
      Class method will return computed r2 score.

    Raises:
      None.
    """
    raise NotImplementedError(
        "This notebook/CL focuses on classification."
        "Next iteration will have complete regression"
        "benchmarking capabilities added."
    )

  def export_benchmarking_results(
      self,
      path_to_save: str,
      serialization_preference: Literal["json", "pkl"] = "pkl",
  ) -> None:
    """Class method for exporting benchmarking instance and results.

    Args:
      path_to_save: A string containing the desired path to save results.
      serialization_preference: A string containing the desired serialization
        type. Currently we support only pickle ('pkl') or JSON ('json').

    Returns:
      None.

    Raises:
    ValueError: If URL to gSheets is not provided when exporting to gSheets has
      been set to True.
    ValueError: If serialization type is not `pkl` or `json`.
    """

    if self.verbose:
      print(f"==> Exporting results to CNS located @: {path_to_save}")

    if serialization_preference.lower() == "pkl":
      self._serialize_results_with_pickle(path_to_save)
    elif serialization_preference.lower() == "json":
      self._serialize_results_with_json(path_to_save)
    else:
      raise ValueError(
          f"Unknown serialization type: {serialization_preference.lower()}."
          "Currently we support only 'pkl' or 'json' as"
          " serialization type."
      )
    if self.verbose:
      print(">-< Done. ")

  def generate_synthetic_data(
      self,
      real_data: np.ndarray,
      real_labels: np.array,
      number_of_synthetics_per_label: Dict[int, int],
      number_of_neighbors: int = 5,
      random_seed: int = 0,
  ) -> tuple[np.ndarray, np.ndarray]:
    """Performs SMOTE upsampling to generate synthetic data per label.

    Args:
      real_data: A numpy array that is the real data we want to use.
      real_labels (numpy.ndarray): A numpy array containing ture labels.
      number_of_synthetics_per_label (dict): Dictionary containing the desired
        number of synthetics per label.
      number_of_neighbors: An integer indicating the desired number of neighbors
        we want to use when generating synthetic samples.
      random_seed: An integer indicating the random seed.

    Returns:
      Two numpy arrays, the upsampled data matrix and corresponding labels.

    Raises:
      ValueError: If
    """
    np.random.seed(random_seed)

    def _find_knn(x: np.ndarray, x_label: np.ndarray, n_neighbors: int):
      """Finds the k-nearest neighbors of x in X_label."""
      distances = np.linalg.norm(x_label - x, axis=1)
      return np.argsort(distances)[:n_neighbors]

    def _generate_synthetic_sample(
        x: np.ndarray,
        neighbor: np.ndarray,
        noise_distribution: Literal["uniform", "gaussian"] = "uniform",
    ):
      """Generates a synthetic sample between x and its neighbor."""
      if noise_distribution == "gaussian":
        return x + np.random.normal(0, 1) * (neighbor - x)
      elif noise_distribution == "uniform":
        return x + np.random.uniform(0, 1) * (neighbor - x)
      else:
        raise ValueError(f"Unknown noise distribution: {noise_distribution}")

    # Upsample each label to the desired number of samples
    x_upsampled = []
    y_upsampled = []
    for label, n_samples in number_of_synthetics_per_label.items():
      x_label = real_data[real_labels == label]
      for _ in range(n_samples):
        # Select a random sample from the label
        idx = np.random.choice(len(x_label))
        x = x_label[idx]
        neighbors_idx = _find_knn(x, x_label, int(number_of_neighbors))
        # Select a random neighbor.
        neighbor = x_label[np.random.choice(neighbors_idx)]
        synthetic_sample = _generate_synthetic_sample(
            x, neighbor, noise_distribution="uniform"
        )

        x_upsampled.append(synthetic_sample)
        y_upsampled.append(label)

    x_upsampled = np.array(x_upsampled)
    y_upsampled = np.array(y_upsampled)

    return x_upsampled, y_upsampled

  def get_complete_prediction_dataframe(self) -> pd.DataFrame:
    """A getter method to return the complete prediction dataframe."""
    return self.prediction_results_dataframe

  def _run_downstream_task(self) -> None:
    """Internal class for performing classification or regression with XGBoost.

    Raises:
      ValueError: If model parameters are not set through class attribute.
      ValueError: If provided cross validation type is not `loocv` or `kfold`.
    """
    if not self.model_parameters:
      raise ValueError(
          "Model parameters need to be either provided or found"
          "through grid search method."
      )

    if self.true_labels_through_cv or self.true_labels_through_cv:
      print(
          "--> Warning: It seems like a previous downstream task set labels."
          " Current run will override these results."
      )
    # We are adding this specific case here to avoid getting XGBoost warnings
    # and slowing down the process.
    if self.task_code == 1 and self.ml_model_name == "xgboost":
      self.model = self._get_sklearn_model()(
          **self.model_parameters, eval_metric="mlogloss", verbosity=0
      )
    else:
      self.model = self._get_sklearn_model()(**self.model_parameters)

    true_labels_through_cv = []
    predicted_labels_through_cv = []
    predicted_participant_ids = []
    split_indices = []
    predicted_probabilities = []

    if self.cross_val_mode.lower() == "loocv":
      cv = model_selection.LeaveOneOut()
    elif self.cross_val_mode.lower() == "kfold":
      cv = model_selection.KFold(
          n_splits=5, random_state=self._random_seed, shuffle=True
      )
    else:
      raise ValueError(
          f"Unknown CV type: {self.cross_val_mode}. Please make "
          "sure to provide either 'loocv' or 'kfold' as CV type."
      )

    for iteration, (train_index, test_index) in enumerate(
        cv.split(self.data_points)
    ):
      x_train = self.data_points[train_index, :]
      y_train = self.targets[np.array(train_index)]

      if self.augment_training_data:
        if self.verbose:
          print(f"    -> Augmenting training data @ iteration: {iteration}.")
        if not self.number_of_synthetics_per_label:
          print(
              "        --> Did not detect number of generation per class.\n"
              "        --> Resorting to default values (100 synth per class)."
          )
          self.number_of_synthetics_per_label = {
              i: 100 for i in np.unique(y_train)
          }

        x_upsampled, y_upsampled = self.generate_synthetic_data(
            x_train,
            y_train,
            self.number_of_synthetics_per_label,
            self.n_neighb_for_synth,
            self._random_seed,
        )

        if self.verbose and self.augmentation_verbose:
          print(
              "        --> Size of training data before augmentation: "
              f"{x_train.shape}."
          )

        x_train = np.concatenate([x_train, x_upsampled], axis=0)
        y_train = np.concatenate([y_train, y_upsampled], axis=0)
        if self.verbose and self.augmentation_verbose:
          print(
              "        --> Size of training data after augmentation: "
              f"{x_train.shape}."
          )
        # Checking that the number of labels and datapoints match after
        # augmentation.
        if len(x_train) != len(y_train):
          raise ValueError(
              "Number of labels and datapoints do not match after "
              "augmentation. Check augmentation logic and try "
              "again."
          )

      x_test = self.data_points[test_index, :]
      y_test = self.targets[test_index]
      test_set_participant_ids = self.participant_ids[test_index]

      self.model.fit(x_train, y_train)
      y_predicted = self.model.predict(x_test)

      if self.cross_val_mode.lower() == "loocv":
        true_labels_through_cv.append(float(y_test[0]))
        predicted_labels_through_cv.append(float(y_predicted[0]))
        predicted_participant_ids.append(test_set_participant_ids)
        split_indices.append(iteration)
        predicted_probabilities.append(
            self.model.predict_proba(x_test).squeeze(axis=0)
        )
      else:
        true_labels_through_cv.extend(y_test)
        predicted_labels_through_cv.extend(y_predicted)
        predicted_participant_ids.extend(test_set_participant_ids)
        split_indices.extend([iteration] * len(y_test))
        predicted_probabilities.extend(self.model.predict_proba(x_test))

      self.true_labels_through_cv = np.array(true_labels_through_cv)
      self.predicted_labels_through_cv = np.array(predicted_labels_through_cv)
      self.predicted_participant_ids = np.array(predicted_participant_ids)
      self.split_indices = np.array(split_indices)
      self.neg_log_probability = -1 * np.log(np.array(predicted_probabilities))
      self.probability = np.array(predicted_probabilities)
      self.prediction_results_dataframe = pd.DataFrame(
          columns=["participant_id"], data=self.predicted_participant_ids
      )
      self.prediction_results_dataframe["Split Iteration"] = self.split_indices
      self.prediction_results_dataframe["True Labels"] = (
          self.true_labels_through_cv
      )
      self.prediction_results_dataframe["Predicted Labels"] = (
          self.predicted_labels_through_cv
      )
      # Create column names based on number of classes and add to the dataframe.
      probability_names = [
          f"Class_{i}_Probability" for i in range(self.probability.shape[1])
      ]

      self.prediction_results_dataframe[probability_names] = self.probability

  def _run_grid_search(self) -> None:
    """Internal class method for running grid search."""

    start_time = time.time()
    # Finding best parameters for XGBoost through grid search if no
    # pre-determined parameters are provided.
    if self.parameter_grid is None:
      print("    -> No parameter grid was provided. Resorting to default grid.")
      self.parameter_grid = self._get_default_parameter_grid()

    model = self._get_sklearn_model()
    # We are adding this specific case here to avoid getting XGBoost warnings
    # and slowing down the process.
    if self.task_code == 1 and self.ml_model_name == "xgboost":
      grid_search_model = model(eval_metric="mlogloss")
    else:
      grid_search_model = model()

    grid_search_model = model_selection.GridSearchCV(
        grid_search_model,
        self.parameter_grid,
        error_score="raise",
        scoring=self.grid_search_metric,
    )

    grid_search_model.fit(self.data_points, self.targets)

    if self.verbose:
      print(f"    -> Grid Search took {time.time() - start_time} seconds.")
      print(f"    -> Best score: {grid_search_model.best_score_}")
      print(f"    -> Best params:\n {grid_search_model.best_params_}")

    self.model_parameters = grid_search_model.best_params_

  def _serialize_results_with_pickle(self, file_path: str) -> None:
    """Internal class method for serializing results with pickle."""
    if self.verbose:
      print(f"    -> Pickling object to: {file_path}")
    with open(file_path, "wb") as outfile:
      pickle.dump(self, outfile)
    print(f">-< Done. Wrote to {file_path}")

  def _serialize_results_with_json(self, file_path: str) -> None:
    """Internal class method for serializing results with JSON."""
    if self.verbose:
      print(f"    -> Saving object in JSON format to: {file_path}")
    with open(file_path, "wb") as outfile:
      json.dump(self, outfile)
    print(f">-< Done. Wrote JSON to {file_path}")

  def _get_sklearn_model(self):
    """Getter ML model getter based on the input model.

    Returns:
      Either an instance of XGBoost or scikit-learn ML methods based on the
      input model or task (classification or regression).
    """

    model_map = {
        "xgboost": (xgboost.XGBClassifier, xgboost.XGBRegressor),
        "random_forest": (
            ensemble.RandomForestClassifier,
            ensemble.RandomForestRegressor,
        ),
        "svm": (svm.SVC, svm.SVR),
        "knn": (neighbors.KNeighborsClassifier, neighbors.KNeighborsRegressor),
        "logistic_regression": (linear_model.LogisticRegression, None),
        # Logistic regression doesn't have a direct regressor counterpart
        "decision_tree": (
            tree.DecisionTreeClassifier,
            tree.DecisionTreeRegressor,
        ),
    }

    if self.ml_model_name.lower() in model_map:
      # `task_code` being 1 indicates that we have a classification problem.
      if self.task_code == 1:
        return model_map[self.ml_model_name.lower()][0]
      # `task_code` not being 1 (being 2) indicates a regression task.
      else:
        return model_map[self.ml_model_name.lower()][1]
    else:
      raise ValueError(f"Unsupported ML model: {self.ml_model_name.lower()}")

  def _get_default_parameter_grid(self):
    """Getter for default grid search parameters based on input ML model."""

    if self.ml_model_name.lower() == "xgboost":
      return XGBOOST_DEFAULT_PARAM_GRID

    elif self.ml_model_name.lower() == "random_forest":
      return RANDOM_FOREST_DEFAULT_PARAM_GRID

    elif self.ml_model_name.lower() == "svm":
      return SVM_DEFAULT_PARAM_GRID

    elif self.ml_model_name.lower() == "knn":
      return KNN_DEFAULT_PARAM_GRID

    elif self.ml_model_name.lower() == "logistic_regression":
      return LOGISTIC_REG_DEFAULT_PARAM_GRID

    elif self.ml_model_name.lower() == "decision_tree":
      return DECISION_TREE_DEFAULT_PARAM_GRID
    else:
      raise ValueError(
          f"Unsupported ML model: {self.ml_model_name.lower}. "
          "Current default grids are available for 'xgboost', "
          "'random_forest', 'svm', 'knn', 'logistic_regression', "
          "and 'decision_tree' models."
      )

  def _flatten_dataframe(
      self, dataframe: pd.DataFrame, column_to_pivot_on: str = "class"
  ) -> pd.DataFrame:
    """Internal class method for flattening evaluation metric predictions."""

    columns_to_keep_values = dataframe.columns.tolist()
    columns_to_keep_values.remove(column_to_pivot_on)
    dataframe["pivot_index"] = 0
    dataframe_pivoted = dataframe.pivot(
        index="pivot_index",
        columns=column_to_pivot_on,
        values=columns_to_keep_values,
    )
    # Removing the temporary column in case multiple calls are done to the same
    # object.
    try:
      del dataframe_pivoted["pivot_index"]
    except KeyError:
      pass

    dataframe_pivoted.columns = [
        f"{metric.upper()} of {label}"
        for metric, label in dataframe_pivoted.columns
    ]

    return dataframe_pivoted.iloc[0].to_frame().T

### Utility function for manually computing multi-class metrics

In [ ]:
def classification_metrics(
    confusion_mat: np.ndarray = None,
    predictions_dataframe: pd.DataFrame = None,
    col_name_for_ground_truth: str = None,
    col_name_for_predictions: str = None,
    labels_to_names_dict: Dict[int, str] = None,
):
  """Utility function for manually computing multi-class metrics.

  Note: This method aims to provide a manual way of testing for the computed
  metrics, as well as providing a way for quicly computing various metrics
  on a confusion matrix, without the need to go through the pipeline.

  Args:
    confusion_mat: A NumPy array representing the confusion matrix. If None, a
      dataframe consisting of predictions and ground truth should be provided.
    predictions_dataframe: A Pandas DataFrame containing predictions and ground
      truth predictions. If None, the confusion matrix should be provided.
    col_name_for_ground_truth: Name of the column in the above dataframe that
      contains true labels.
    col_name_for_predictions: Name of the column in the above dataframe that
      contains model predictions.
    labels_to_names_dict: A dictionary mapping labels to names for better
      interpretability of the results dataframe.

  Returns:
    A Pandas DataFrame containing the sensitivity, specificity, precision,
    and false positive rate values for each class.

  Raises:
    ValueError: If neither `confusion_mat` or `predictions_dataframe` are
      provided.
  """
  if confusion_mat is None and predictions_dataframe is None:
    raise ValueError(
        'Please provide either `confusion_mat` or '
        '`predictions_dataframe` in order to compute accuracy '
        'metrics.'
    )

  if confusion_mat is None:
    y_true = predictions_dataframe[col_name_for_ground_truth]
    y_pred = predictions_dataframe[col_name_for_predictions]
    confusion_mat = metrics.confusion_matrix(y_true, y_pred)

  num_classes = confusion_mat.shape[0]
  metric_names = [
      'Specificity',
      'Sensitivity',
      'Precision',
      'F1 Score',
      'False Positive Rate',
  ]
  results = []

  for current_class in range(num_classes):
    tp = confusion_mat[current_class, current_class]
    fn = np.sum(confusion_mat[current_class, :]) - tp
    fp = np.sum(confusion_mat[:, current_class]) - tp
    tn = np.sum(confusion_mat) - (tp + fn + fp)

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
    f1_score = (
        (2 * precision * sensitivity) / (precision + sensitivity)
        if (precision + sensitivity) > 0
        else 0
    )

    results.append(
        [specificity, sensitivity, precision, f1_score, false_positive_rate]
    )

  # Create a DataFrame from the results
  results_df = pd.DataFrame(
      results, index=range(num_classes), columns=metric_names
  )

  if labels_to_names_dict:
    results_df['Class'] = results_df.index.map(labels_to_names_dict)
    first_column = results_df.pop('Class')
    results_df.insert(0, 'Class', first_column)

  return results_df

In [ ]:
# @title Regression Targets to Classification Converter (with threshold)

def regression_to_classification(regression_targets: np.ndarray,
                                 thresholds: list) -> np.ndarray:

  """Converts regression targets to classification labels based on user-defined thresholds.

  Args:
    regression_targets: A 1D numpy array of regression targets.
    thresholds: A list of thresholds used to define the classification boundaries.
      The thresholds should be sorted in ascending order.

  Returns:
    A 1D numpy array of classification labels, where each label corresponds to the index
    of the threshold range the original regression target falls into.

  Note:
    - The number of classes will be one more than the number of thresholds.
    - Values below the first threshold will be assigned label 0.
    - Values above the last threshold will be assigned the highest label.
  """

  n_classes = len(thresholds) + 1
  classification_labels = np.zeros_like(regression_targets, dtype=int)

  for i in range(n_classes):
    if i == 0:
      classification_labels[regression_targets < thresholds[i]] = i
    elif i == n_classes - 1:
      classification_labels[regression_targets >= thresholds[i - 1]] = i
    else:
      classification_labels[(regression_targets >= thresholds[i - 1]) & (regression_targets < thresholds[i])] = i

  return classification_labels

### Data Processing and Scoring Utility Functions

In [18]:
def preprocess_data(X_train, X_test, scaler_name="standard"):
    """
    Preprocesses and splits a Pandas DataFrame for neural network training,
    returning the scaler object for inverse transformation.

    Args:
        df: The input Pandas DataFrame.
        target_cols: A list of column names representing the target variables (optional).
                     If not provided, the entire DataFrame is considered for normalization.

    Returns:
        A tuple containing:
            - Train and test DataFrames, potentially with normalized values and split into
              features (X) and targets (y) if target_cols are provided.
            - The fitted StandardScaler object for inverse transformation.
    """
    if scaler_name == "minmax":
      scaler = MinMaxScaler()
    elif scaler_name == "standard":
      scaler = StandardScaler()
    else:
      raise ValueError(f"Unknown scaler name: {scaler_name}")

    x_train_normalized = scaler.fit_transform(X_train)
    x_test_normalized = scaler.transform(X_test)

    return x_train_normalized, x_test_normalized

def inverse_transform_data(df, scaler, target_cols=None):
    """
    Applies inverse transformation to normalized data using the provided StandardScaler.

    Args:
        df: The DataFrame containing normalized data.
        scaler: The fitted StandardScaler object used for normalization.
        target_cols: A list of column names representing the target variables (optional).
                     If provided, these columns will not be inverse transformed.

    Returns:
        A DataFrame with the data transformed back to the original scale.
    """
    if target_cols:
        # Separate features and targets
        X_norm = df.drop(target_cols, axis=1)
        y = df[target_cols]

        # Inverse transform features
        X_orig = pd.DataFrame(scaler.inverse_transform(X_norm), columns=X_norm.columns)

        # Combine inverse transformed features and (unchanged) targets
        df_orig = pd.concat([X_orig, y], axis=1)
    else:
        # Inverse transform the entire DataFrame
        df_orig = pd.DataFrame(scaler.inverse_transform(df), columns=df.columns)

    return df_orig

def evaluate_regression_trials(predicted_targets, true_targets, verbose=True):
  """
  Evaluates regression model performance across multiple trials and prints metrics.

  Args:
      predicted_targets (list): List of lists containing predicted target values for each trial.
      true_targets (list): List of lists containing true target values for each trial.
  """

  mse_scores = []
  mae_scores = []

  # Calculate MSE and MAE for each trial
  for pred, true in zip(predicted_targets, true_targets):
      mse_scores.append(metrics.mean_squared_error(true, pred))
      mae_scores.append(metrics.mean_absolute_error(true, pred))

  # Calculate mean and standard deviation
  mse_mean, mse_std = np.mean(mse_scores), np.std(mse_scores)
  mae_mean, mae_std = np.mean(mae_scores), np.std(mae_scores)

  # Print results
  if verbose:
    print("Mean Squared Error (MSE): {:.4f} +/- {:.4f}".format(mse_mean, mse_std))
    print("Mean Absolute Error (MAE): {:.4f} +/- {:.4f}".format(mae_mean, mae_std))

  # Concatenate predictions and true targets from all trials
  all_predictions = np.concatenate(predicted_targets)
  all_true_targets = np.concatenate(true_targets)

  # Calculate R^2
  r2 = metrics.r2_score(all_true_targets, all_predictions)
  if verbose:
    print("R^2 Score: {:.4f}".format(r2))

  return {'R2_score': r2,
          'MAE_mean': mae_mean,
          'MAE_std': mae_std,
          'MSE_mean': mse_mean,
          'MSE_std': mae_std}


def evaluate_classification_trials(predicted_labels, true_labels,
                                   labels_to_names_dict: dict = None,
                                   verbose:bool = True):
  """
  Calculates various classification metrics across multiple trials, including:
  - Average specificity, sensitivity, precision for each class
  - Area Under the Receiver Operating Characteristic curve (AUROC) for each class
  - Area Under the Precision-Recall curve (AUPRC) for each class

  Handles cases where a class has no predicted instances by manually calculating precision.

  Args:
    predicted_labels: A list of lists containing predicted labels for each trial.
    true_labels: A list of lists containing true labels for each trial.
    labels_to_names_dict: (Optional) A dictionary mapping class labels to descriptive names.

  Returns:
    A dictionary containing:
      - class_names: A list of unique class names.
      - avg_specificity: A NumPy array with the average specificity for each class.
      - std_specificity: A NumPy array with the standard deviation of specificity for each class.
      - avg_sensitivity: A NumPy array with the average sensitivity for each class.
      - std_sensitivity: A NumPy array with the standard deviation of sensitivity for each class.
      - avg_precision: A NumPy array with the average precision for each class.
      - std_precision: A NumPy array with the standard deviation of precision for each class.
      - avg_auroc: A NumPy array with the average AUROC for each class.
      - std_auroc: A NumPy array with the standard deviation of AUROC for each class.
      - avg_auprc: A NumPy array with the average AUPRC for each class.
      - std_auprc: A NumPy array with the standard deviation of AUPRC for each class.
  """

  # Check if the input lists have the same length
  if len(predicted_labels) != len(true_labels):
    raise ValueError("predicted_labels and true_labels must have the same length.")

  # Get unique class names
  try:
    class_names = np.unique(np.concatenate(true_labels))
  except ValueError:
    class_names = np.unique(true_labels)

  # Initialize arrays to store metrics
  num_classes = len(class_names)
  avg_specificity = np.zeros(num_classes)
  std_specificity = np.zeros(num_classes)
  avg_sensitivity = np.zeros(num_classes)
  std_sensitivity = np.zeros(num_classes)
  avg_precision = np.zeros(num_classes)
  std_precision = np.zeros(num_classes)
  avg_auroc = np.zeros(num_classes)
  std_auroc = np.zeros(num_classes)
  avg_auprc = np.zeros(num_classes)
  std_auprc = np.zeros(num_classes)

  # Iterate over classes
  for i, class_name in enumerate(class_names):
    # Collect metrics for each trial
    specificity_list = []
    sensitivity_list = []
    precision_list = []
    auroc_list = []
    auprc_list = []

    for trial_predicted, trial_true in zip(predicted_labels, true_labels):
      # One-hot encode labels for the current class
      trial_predicted_onehot = (trial_predicted == class_name).astype(int)
      trial_true_onehot = (trial_true == class_name).astype(int)

      # Calculate metrics for the current trial and class
      specificity_list.append(metrics.recall_score(trial_true_onehot, trial_predicted_onehot, pos_label=0))
      sensitivity_list.append(metrics.recall_score(trial_true_onehot, trial_predicted_onehot, pos_label=1))

      # Manually calculate precision
      tp = np.sum((trial_predicted_onehot == 1) & (trial_true_onehot == 1))  # True positives
      fp = np.sum((trial_predicted_onehot == 1) & (trial_true_onehot == 0))  # False positives

      if (tp + fp) == 0:
          precision_list.append(0.0) # Set precision to 0 when there are no positive predictions
      else:
          precision_list.append(tp / (tp + fp))

      # Calculate AUROC and AUPRC
      fpr, tpr, _ = metrics.roc_curve(trial_true_onehot, trial_predicted_onehot)
      auroc_list.append(metrics.auc(fpr, tpr))

      precision, recall, _ = metrics.precision_recall_curve(trial_true_onehot, trial_predicted_onehot)
      auprc_list.append(metrics.auc(recall, precision))

    # Calculate average and standard deviation for the class
    avg_specificity[i] = np.mean(specificity_list)
    std_specificity[i] = np.std(specificity_list)
    avg_sensitivity[i] = np.mean(sensitivity_list)
    std_sensitivity[i] = np.std(sensitivity_list)
    avg_precision[i] = np.mean(precision_list)
    std_precision[i] = np.std(precision_list)
    avg_auroc[i] = np.mean(auroc_list)
    std_auroc[i] = np.std(auroc_list)
    avg_auprc[i] = np.mean(auprc_list)
    std_auprc[i] = np.std(auprc_list)

  # Print results for each class
  if verbose:
    for i, class_name in enumerate(class_names):
      if labels_to_names_dict is None:
        print(f"==> Class: {class_name}")
      else:
        print(f"==> Class: {labels_to_names_dict[class_name]}")

      print(f"    -> Specificity: {avg_specificity[i]:.4f} +/- {std_specificity[i]:.4f}")
      print(f"    -> Sensitivity: {avg_sensitivity[i]:.4f} +/- {std_sensitivity[i]:.4f}")
      print(f"    -> Precision:   {avg_precision[i]:.4f} +/- {std_precision[i]:.4f}")
      print(f"    -> AUROC:        {avg_auroc[i]:.4f} +/- {std_auroc[i]:.4f}")
      print(f"    -> AUPRC:        {avg_auprc[i]:.4f} +/- {std_auprc[i]:.4f}")

    # Calculate and print false positive rate for the class
    false_positive_rate_list = []
    for trial_predicted, trial_true in zip(predicted_labels, true_labels):
      # One-hot encode labels for the current class
      trial_predicted_onehot = (trial_predicted == class_name).astype(int)
      trial_true_onehot = (trial_true == class_name).astype(int)

      # Calculate false positive rate for the current trial and class
      fp = np.sum((trial_predicted_onehot == 1) & (trial_true_onehot == 0))  # False positives
      tn = np.sum((trial_predicted_onehot == 0) & (trial_true_onehot == 0))  # True negatives
      false_positive_rate_list.append(fp / (fp + tn) if (fp + tn) > 0 else 0)  # Avoid division by zero

    avg_false_positive_rate = np.mean(false_positive_rate_list)
    std_false_positive_rate = np.std(false_positive_rate_list)
    print(f"    -> False Positive Rate: {avg_false_positive_rate:.4f} +/- {std_false_positive_rate:.4f}")

  # Return results for the Insulin Resistant class
  return {'specificity_mean': avg_specificity[-1],
          'specificity_std': std_specificity[-1],
          'sensitivity_avg': avg_sensitivity[-1],
          'sensitivity_std': std_sensitivity[-1],
          'precision_avg': avg_precision[-1],
          'precision_std': std_precision[-1],
          'auroc_avg': avg_auroc[-1],
          'auroc_std': std_auroc[-1],
          'auprc_avg': avg_auprc[-1],
          'auprc_std': std_auprc[-1]}

def calculate_mean_std_across_trials(dataframes: list,
                                     labels_to_names_dict: dict=None,
                                     verbose:bool=True) -> pd.DataFrame:
  """
  Calculates the mean and standard deviation of classification metrics across multiple trials
  and prints the results for each metric per class.

  Args:
    dataframes: A list of Pandas DataFrames, where each DataFrame contains the following columns:
      - 'Specificity'
      - 'Sensitivity'
      - 'Precision'
      - 'F1 Score'
      - 'False Positive Rate'
      Each row in the DataFrames represents a class, and each column represents a trial.
    labels_to_names_dict: (Optional) A dictionary mapping class labels to descriptive names.

  Returns:
    A Pandas DataFrame containing the mean and standard deviation for each metric across all trials.
    The DataFrame will have the same rows (classes) as the input DataFrames,
    and the following columns for each metric:
      - '{Metric Name} Mean'
      - '{Metric Name} Std'
  """

  # Concatenate dataframes horizontally for each metric
  all_specificity = pd.concat([df['Specificity'] for df in dataframes], axis=1)
  all_sensitivity = pd.concat([df['Sensitivity'] for df in dataframes], axis=1)
  all_precision = pd.concat([df['Precision'] for df in dataframes], axis=1)
  all_f1 = pd.concat([df['F1 Score'] for df in dataframes], axis=1)
  all_fpr = pd.concat([df['False Positive Rate'] for df in dataframes], axis=1)

  # Calculate mean and standard deviation for each metric
  accuracy_dict = {
      'Specificity Mean': all_specificity.mean(axis=1),
      'Specificity Std': all_specificity.std(axis=1),
      'Sensitivity Mean': all_sensitivity.mean(axis=1),
      'Sensitivity Std': all_sensitivity.std(axis=1),
      'Precision Mean': all_precision.mean(axis=1),
      'Precision Std': all_precision.std(axis=1),
      'F1 Score Mean': all_f1.mean(axis=1),
      'F1 Score Std': all_f1.std(axis=1),
      'False Positive Rate Mean': all_fpr.mean(axis=1),
      'False Positive Rate Std': all_fpr.std(axis=1),
  }
  mean_std_df = pd.DataFrame(accuracy_dict)

  ir_class_accuracy_dict = {}
  # Print results for each metric per class
  for index, row in mean_std_df.iterrows():
    if labels_to_names_dict is None:
      class_name = index
    else:
      class_name = labels_to_names_dict[index]

    if class_name == "Insulin Resistant":
      ir_class_accuracy_dict = row.to_dict()

    if verbose:
      print(f"==> Class: {class_name}")
      print(f"    -> Specificity: {row['Specificity Mean']:.4f} +/- {row['Specificity Std']:.4f}")
      print(f"    -> Sensitivity: {row['Sensitivity Mean']:.4f} +/- {row['Sensitivity Std']:.4f}")
      print(f"    -> Precision:   {row['Precision Mean']:.4f} +/- {row['Precision Std']:.4f}")
      print(f"    -> F1 Score:    {row['F1 Score Mean']:.4f} +/- {row['F1 Score Std']:.4f}")
      print(f"    -> False Positive Rate: {row['False Positive Rate Mean']:.4f} +/- {row['False Positive Rate Std']:.4f}")

  return mean_std_df, ir_class_accuracy_dict

# Study code



In [ ]:
# @title Collect Data by Number of Days

def aggregate_summarize_digital_biomarker(data, column_of_interest, prefix, number_of_days):
  """Aggregates and summarizes digital biomarker data for each participant over a specified number of days.

  Args:
      data (pd.DataFrame): The input DataFrame containing the digital biomarker data.
      column_of_interest (str): The name of the column containing the digital biomarker values.
      prefix (str): The prefix to add to the aggregated column names.
      number_of_days (int): The number of days to consider for aggregation.

  Returns:
      pd.DataFrame: A DataFrame containing the aggregated digital biomarker data for each participant.
  """
  # Sort data by participant_id and activity_date
  data = data.sort_values(['participant_id', 'activity_date'])

  # Group by participant_id
  grouped_data = data.groupby('participant_id')

  summary_data = []

  # Iterate over each participant's data
  for participant_id, participant_data in grouped_data:
    # Select the most recent 'number_of_days' of data
    recent_data = participant_data.tail(number_of_days)
    # Calculate summary statistics for the selected data
    participant_summary = recent_data[column_of_interest].agg(['mean', 'median', 'std'])
    # Rename columns for the Series
    participant_summary.index = prefix + participant_summary.index

    # Convert to DataFrame and assign participant_id
    participant_summary = participant_summary.to_frame().transpose()
    participant_summary['participant_id'] = participant_id

    summary_data.append(participant_summary)

  # Combine summary data for all participants
  summary_data = pd.concat(summary_data, axis=0).reset_index(drop=True)

  return summary_data

def aggregate_markers_for_every_n_days(data, column_of_interest,
                                       prefix, number_of_days,
                                       min_days_required=91):
  """
  Aggregates and summarizes digital biomarker data for each participant over
  multiple discontiguous periods of specified length, filtering participants
  based on a minimum number of days required and taking only the last portion
  of their data. Handles individual date ranges for each participant.

  Args:
    data (pd.DataFrame): The input DataFrame containing the digital biomarker data.
    column_of_interest (str): The name of the column containing the digital biomarker values.
    prefix (str): The prefix to add to the aggregated column names.
    number_of_days (int): The number of days in each period for aggregation.
    min_days_required (int): The minimum number of days required for a
                              participant to be included in the analysis.

  Returns:
    list: A list of DataFrames, each containing the aggregated digital
          biomarker data for participants over one period.
  """
  data = data.sort_values(['participant_id', 'activity_date'])

  summary_dataframes = []

  # 1. Filter Participants Based on Minimum Days:
  valid_participants = data.groupby('participant_id')['activity_date'].count() >= min_days_required
  valid_participants = valid_participants[valid_participants].index.tolist()
  data = data[data['participant_id'].isin(valid_participants)]

  # 2. Determine Maximum Possible Periods:
  max_periods = min_days_required // number_of_days

  # 3. Iterate Through Periods:
  for period_index in range(max_periods):
    period_data_list = []

    # 4. Iterate Through Participants:
    for participant_id in valid_participants:
      participant_data = data[data['participant_id'] == participant_id]

      # 5. Calculate End Date Index:
      end_date_index = len(participant_data) - (period_index * number_of_days)

      # 6. Calculate Start Date Index:
      start_date_index = end_date_index - number_of_days

      # 7. Slice Data for the Period (if applicable):
      if start_date_index >= 0:
        participant_period_data = participant_data.iloc[start_date_index:end_date_index]
        # Correctly aggregate with 'participant_id'
        participant_period_data = participant_period_data.groupby('participant_id')[column_of_interest].agg(['mean', 'median', 'std']).reset_index()
        period_data_list.append(participant_period_data)
      else:
        # Create a DataFrame with NaN values for missing periods
        participant_period_summary = pd.DataFrame({
            'participant_id': [participant_id],
            (prefix + 'mean'): [np.nan],
            (prefix + 'median'): [np.nan],
            (prefix + 'std'): [np.nan]
        })
        period_data_list.append(participant_period_summary)

    # 8. Combine Period Data for All Participants:
    period_data = pd.concat(period_data_list, ignore_index=True)

    # 9. Aggregate for the Period:
    # period_summary = period_data.groupby('participant_id')[column_of_interest].agg(['mean', 'median', 'std'])
    period_data.columns = [prefix + col if col != 'participant_id' else col for col in period_data.columns]
    summary_dataframes.append(period_data)

  return summary_dataframes

In [ ]:
# @title Reading in Our Data (Averaged Over All Days)
training_data_aggregated_all_days['hypertension_binary']=(
                  training_data_aggregated_all_days['hypertension'].astype(int))

training_data_aggregated_all_days

,participant_id,RHR_FreeLiving_mean,RHR_FreeLiving_median,RHR_FreeLiving_std,HRV_FreeLiving_mean,HRV_FreeLiving_median,HRV_FreeLiving_std,STEPS_Daily_mean,STEPS_Daily_median,STEPS_Daily_std,...,framingham_risk_category,optimal_framingham_risk,framingham30_risk,framingham30_risk_category,optimal_framingham30_risk,ascvd_risk,ascvd_risk_category,optimal_ascvd_risk,apri,hypertension_binary
0,12591,65.509615,65.0,1.960948,25.452000,25.0305,4.018824,7329.278846,6557.5,3087.041901,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.350877,1
1,12597,49.201923,49.0,1.623379,15.998155,15.9660,2.052595,5357.019231,4956.0,2354.257334,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.503513,1
2,12599,65.168421,64.0,4.636302,42.677238,43.4890,10.082448,9036.231579,8172.0,4653.169485,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.214918,0
3,12612,71.080000,71.0,1.796068,44.996636,46.0220,7.762804,10998.481481,9806.0,5084.491732,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.220629,0
4,12614,65.200000,64.0,4.300286,22.555556,22.2240,5.748754,5953.837438,5469.0,3471.700982,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.612245,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,199756,68.432099,69.0,3.625096,22.274354,22.0030,6.682359,7019.839506,7004.0,2800.065485,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.171086,0
1169,199757,65.696078,65.0,3.522732,11.017429,10.9355,1.574011,11991.941176,10074.0,7029.545331,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.249653,1
1170,199758,64.527132,65.0,2.334240,28.951650,27.5440,7.033047,5076.480620,5088.0,2297.668607,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.151313,0
1171,199759,69.549451,69.0,3.029373,14.461491,14.5800,3.405676,5321.453608,4643.0,4216.633141,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.234962,0


In [19]:
# @title Define experiments

EXP_1 = {'name': "EXP_1",
'description': "Fitbit + Demographics",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std'],
'label': "homa_ir"}

EXP_2 = {'name': "EXP_2",
'description': "Fitbit subset + Demographics",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'STEPS_Daily_mean', 'SLEEP_Duration_mean'],
'label': "homa_ir"}

EXP_3 = {'name': "EXP_3",
'description': "Fitbit subset",
'features': ['participant_id',
'RHR_FreeLiving_mean', 'STEPS_Daily_mean', 'SLEEP_Duration_mean'],
'label': "homa_ir"}

EXP_4 = {'name': "EXP_4",
'description': "Demographics",
'features': ['participant_id', 'bmi','age'],
'label': "homa_ir"}

EXP_5 = {'name': "EXP_5",
'description': "Fitbit + Demographics + HbA1c",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std',
'hba1c'],
'label': "homa_ir"}

EXP_6 = {'name': "EXP_6",
'description': "Demographics + HbA1c",
'features': ['participant_id', 'bmi','age',
'hba1c'],
'label': "homa_ir"}

EXP_7 = {'name': "EXP_7",
'description': "HbA1c",
'features': ['participant_id',
'hba1c'],
'label': "homa_ir"}

EXP_8 = {'name': "EXP_8",
'description': "Demographics + Fasting Glucose",
'features': ['participant_id', 'bmi','age',
'glucose'],
'label': "homa_ir"}

EXP_9 = {'name': "EXP_9",
'description': "Fasting Glucose",
'features': ['participant_id',
'glucose'],
'label': "homa_ir"}

EXP_10 = {'name': "EXP_10",
'description': "Fitbit + Demographics + Fasting Glucose",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std',
'glucose'],
'label': "homa_ir"}

EXP_11 = {'name': "EXP_11",
'description': "Fitbit + Demographics + Lipid Panel + Metabolic Panel",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt'],
'label': "homa_ir"}


EXP_12 = {'name': "EXP_12",
'description': "Demographics + Lipid Panel + Metabolic Panel",
'features': ['participant_id', 'bmi','age',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt'],
'label': "homa_ir"}


EXP_13 = {'name': "EXP_13",
'description': "Demographics + Hypertension",
'features': ['participant_id', 'bmi','age',
'hypertension_binary'],
'label': "homa_ir"}

EXP_14 = {'name': "EXP_14",
'description': "Fitbit + Demographics + Hypertension",
'features': ['participant_id', 'bmi','age',
'hypertension_binary',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std'],
'label': "homa_ir"}

EXP_15 = {'name': "EXP_15",
'description': "Fitbit + Demographics + Fasting Glucose + Hypertension",
'features': ['participant_id', 'bmi','age', 'hypertension_binary',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std',
'glucose'],
'label': "homa_ir"}


EXP_16 = {'name': "EXP_16",
'description': "Fitbit + Demographics + Lipid Panel + Metabolic Panel + Hypertension",
'features': ['participant_id', 'bmi','age', 'hypertension_binary',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt'],
'label': "homa_ir"}

EXP_17 = {'name': "EXP_17",
'description': "Lipid Panel + Metabolic Panel",
'features': ['participant_id',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt'],
'label': "homa_ir"}

EXP_18 = {'name': "EXP_18",
'description': "Fitbit + Demographics + Lipid Panel + Metabolic Panel + HbA1c",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt',
'hba1c'],
'label': "homa_ir"}


EXP_19 = {'name': "EXP_19",
'description': "Lipid Panel",
'features': ['participant_id',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl'],
'label': "homa_ir"}


EXP_20 = {'name': "EXP_20",
'description': "Metabolic Panel",
'features': ['participant_id', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt'],
'label': "homa_ir"}


EXP_21 = {'name': "EXP_21",
'description': "Fitbit + Demographics + Lipid Panel",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl'],
'label': "homa_ir"}


EXP_22 = {'name': "EXP_22",
'description': "Fitbit + Demographics + Metabolic Panel",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt'],
'label': "homa_ir"}



EXP_23 = {'name': "EXP_23",
'description': "Demographics + Lipid Panel",
'features': ['participant_id', 'bmi','age',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl'],
'label': "homa_ir"}

EXP_24 = {'name': "EXP_24",
'description': "Demographics + Metabolic Panel",
'features': ['participant_id', 'bmi','age', 'glucose', 'bun',
'creatinine', 'egfr', 'sodium', 'potassium', 'chloride', 'co2', 'calcium',
'total protein', 'albumin', 'globulin', 'albumin/globulin', 'total bilirubin',
'alp', 'ast', 'alt'],
'label': "homa_ir"}

EXP_25 = {'name': "EXP_25",
'description': "Fasting Insulin",
'features': ['participant_id', 'insulin'],
'label': "homa_ir"}

EXP_26 = {'name': "EXP_26",
'description': "Fasting Glucose + Fasting Insulin",
'features': ['participant_id', 'glucose', 'insulin'],
'label': "homa_ir"}


EXP_27 = {'name': "EXP_27",
'description': "Fitbit + Demographics + Fasting Glucose + Lipid Panel",
'features': ['participant_id', 'bmi','age',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',
'SLEEP_Duration_std', 'glucose',
'total cholesterol',
'hdl', 'triglycerides', 'ldl', 'chol/hdl', 'non hdl'],
'label': "homa_ir"}


EXP_28 = {'name': "EXP_28",
'description': "Fitbit",
'features': ['participant_id',
'RHR_FreeLiving_mean', 'RHR_FreeLiving_median',
'RHR_FreeLiving_std', 'HRV_FreeLiving_mean', 'HRV_FreeLiving_median',
'HRV_FreeLiving_std', 'STEPS_Daily_mean', 'STEPS_Daily_median',
'STEPS_Daily_std', 'SLEEP_Duration_mean', 'SLEEP_Duration_median',],
'label': "homa_ir"}

In [20]:
# @title Dataloader Definition for Training Masked AE

class CGMDataset(Dataset):
  """
  Dataset class for blood test data.
  """
  def __init__(self, blood_and_digital_markers:np.ndarray, labels:np.array):
    # Load data from the specified path (modify according to your data format)
    if len(blood_and_digital_markers) == 0:
      raise ValueError('Blood and/or digital markers must be non-empty')
    if len(labels) == 0:
      raise ValueError('Labels cannot be empty.')
    if len(blood_and_digital_markers) != len(labels):
      raise ValueError(
                    'Blood and/or digital markers must have the same length.')
    self.data = blood_and_digital_markers
    self.labels = labels

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    # Return a single blood test sample with appropriate processing
    return self.data[idx], self.labels[idx]

In [ ]:
# @title Ablation Study Function and Parameter Grid Generator

import torch.optim as optim
from typing import Literal

def run_dl_ablation_study(
  param_grid,  # Dictionary of parameters and their possible values,
  training_data:pd.DataFrame,
  path_to_save: str = None,
  results_filename: str = None,
  model_save_path: str = None,
  n_splits=5,  # Number of splits for k-fold cross-validation
  random_seeds=[0, 92, 1, 2024, 12121],  # Random seeds for reproducibility,
  return_results: bool = True,
  device:Literal["cpu", "cuda"]=None,
  ae_type: Literal["masked_autoencoder", "autoencoder"] = "masked_autoencoder"
):
  """Performs an ablation study for an XGBoost model with optional representation learning.

  This function iterates through a grid of parameters, trains and evaluates a
  pipeline including a Masked Autoencoder (optional) and an XGBoost model. Results
  are saved to a CSV file.

  Args:
      param_grid (dict): A dictionary where keys are parameter names (str) and values
          are lists of possible values for each parameter. This should include parameters
          for both the MaskedAutoencoder and the XGBoost model.
      path_to_save (str, optional): The path to save the results to. Defaults to None.
      results_filename (str, optional): The filename to save the results to. Defaults to None.
      model_save_path (str, optional): The path to save the model. Defaults to None.
      n_splits (int, optional): The number of splits for k-fold cross-validation. Defaults to 5.
      random_seeds (list, optional): A list of random seeds to use for different iterations of
          cross-validation. Defaults to [0, 92, 1, 2024, 12121].
      return_results (bool, optional): A boolean indicating whether the results should be saved.
          Defaults to True.

  Returns:
      tuple: A tuple containing the DataFrame of results and the classification results if
      return_results is True. Otherwise, returns None.

  Example usage:
      param_grid = {
          'learner': ['l1-l2', 'gbtree'],
          'n_days': [7, 14, 21],
          'experiment_name': ['EXP_1', 'EXP_2', 'EXP_3'],
          'type_of_cross_validation': ['kfold', 'loocv'],
          'use_mae': [True, False], # Whether to use MAE for representation learning
          'latent_space': [32, 64], # Latent space size for the MAE
      }
      run_ablation_study(param_grid)

  Notes:
      -  The 'use_mae' parameter in the param_grid determines whether to use the
      MaskedAutoencoder.
      -  If 'use_mae' is True, the code expects additional parameters in the param_grid
      related to the MaskedAutoencoder, such as 'latent_space'.
      -  The functions `get_n_days_aggregates`, `preprocess_data`,
      `regression_to_classification`, `classification_metrics`,
      `evaluate_regression_trials`, and `evaluate_classification_trials` are assumed
      to be defined elsewhere.
  """

  if device is None:
    if torch.cuda.is_available():
      device = torch.device("cuda")
    else:
      device = torch.device("cpu")

  labels_to_name_dic = {
      0: "Insulin Sensitive",
      1: "Sensitivity Impaired",
      2: "Insulin Resistant",
  }
  results = []

  # Iterate through all combinations of parameters
  param_combinations = generate_parameter_combinations(param_grid)
  for params in tqdm(param_combinations, desc="Ablation Study Progress"):
    learner = params["learner"]
    n_days = params["n_days"]
    experiment_name = params["experiment_name"]
    experiment_id = experiment_name["name"]
    feature_cols = experiment_name["features"]
    target_col = experiment_name["label"]
    type_of_cross_validation = params["type_of_cross_validation"]
    use_mae = params.get("use_mae", True)  # Default to False if not provided
    latent_space = params["latent_space"]

    if learner == "l1-l2":
      kfold_found_params = {'booster': 'gblinear',
                            'learning_rate': 0.19,
                            'n_estimators': 50,
                            'eta':0.1,
                            'reg_alpha': 0,
                            'reg_lambda': 0}

    elif learner == "tree":
      kfold_found_params = {
          "booster": "gbtree",
          "learning_rate": 0.1,
          "max_depth": 1,
          "n_estimators": 85,
          "reg_alpha": 2,
          "reg_lambda": 2,
      }
    else:
      raise ValueError(
          f"Unknown learner: {learner}. Learners must be 'l1-l2' or 'tree'."
      )
    model_dir = ""
    if model_save_path is not None:
      model_dir = os.path.join(model_save_path, f"{params['rl-model']}_{experiment_id}_{type_of_cross_validation}_n_days_{n_days}")

      if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    # Get the training data
    training_df = training_data.copy()
    training_df = training_df.loc[:, feature_cols]
    training_df[target_col] =  training_data[target_col]
    training_df.dropna(inplace=True)
    ordered_participant_ids = []
    accuracy_dfs = []
    all_predictions = []
    all_true_labels = []
    clf_labels_preds = []
    clf_labels_true_labels = []
    if type_of_cross_validation == "kfold":
      cv = model_selection.KFold(n_splits=n_splits)
    else:
      cv = model_selection.LeaveOneOut()

    # Perform cross-validation
    # Add tqdm for k-fold iterations
    if type_of_cross_validation == "kfold":
      cv_iterator = tqdm(
          cv.split(training_df),
          total=n_splits,
          desc=f"  {params['experiment_name']['name']}",
          leave=False,
      )
    else:
      cv_iterator = cv.split(training_df)

    # Perform cross-validation
    print("------")
    for i, (train_index, test_index) in enumerate(cv_iterator):
      if type_of_cross_validation == "loocv":
        random_seeds = np.arange(0, len(training_df) + 1)
      np.random.seed(random_seeds[i])
      torch.manual_seed(random_seeds[i])

      participant_ids = training_df["participant_id"].values
      X_train = training_df.iloc[train_index].drop(columns=["participant_id",
                                                            target_col], axis=1)
      y_train = training_df.iloc[train_index][target_col]
      X_test = training_df.iloc[test_index].drop(columns=["participant_id",
                                                          target_col], axis=1)
      y_test = training_df.iloc[test_index][target_col]
      participant_ids_test = participant_ids[test_index]

      # Representation learning using Masked Autoencoder
      if use_mae:
        # Initialize the MaskedAutoencoder
        X_train, X_test = preprocess_data(X_train, X_test.values)
        train_dataloader = DataLoader(CGMDataset(X_train, y_train.values),
                                      batch_size=512,
                                      shuffle=True)
        num_epochs = 500
        criterion_1 = nn.MSELoss()
        criterion_2 = nn.SmoothL1Loss(reduction='none')
        lambda_ = 0.01
        learning_rate = 0.001
        if ae_type == "masked_autoencoder":
          masking_fraction = 0.75
          dl_model = MaskedAutoencoder(input_dim=X_train.shape[1],
                                  latent_space=latent_space).to(device)
        elif ae_type == "autoencoder":
          masking_fraction = 0
          lambda_ = 0.01
          dl_model = SimpleAutoencoder(input_dim=X_train.shape[1],
                                        latent_space=latent_space).to(device)
        else:
          raise ValueError(f"Unknown ae_type: {ae_type}. 'ae_type' must be "
                            "'masked_autoencoder' or 'autoencoder'.")

        optimizer = optim.Adam(dl_model.parameters(),
                              lr=learning_rate,
                              betas=(0.9, 0.999),
                              eps=1e-12,
                              weight_decay=0.0000,
                              )
        dl_model.train()
        # Train the model
        for epoch in range(num_epochs+1):

          for _, (data, _) in enumerate(train_dataloader):
            inputs = data.float().to(device)
            if ae_type == "masked_autoencoder":
              mask = torch.bernoulli(torch.ones_like(inputs) *
                                     (1-masking_fraction)).float().to(device)
              outputs = dl_model(inputs, mask)
              masked_loss = torch.mean(criterion_2(outputs, inputs) * mask)
              loss = masked_loss + lambda_ * criterion_1(outputs, inputs)
            else:
              outputs = dl_model(inputs)
              loss = criterion_1(outputs, inputs)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # Get encoded representations
        dl_model.eval()
        with torch.no_grad():
          X_train_encoded, _ = dl_model(torch.Tensor(X_train).to(device),
                                         return_encoded = True)
          X_test_encoded, _ = dl_model(torch.Tensor(X_test).to(device),
                                        return_encoded = True)

        # Convert to numpy for XGBoost
        X_train = X_train_encoded.detach().cpu().numpy()
        X_test = X_test_encoded.detach().cpu().numpy()

      # Save the Masked Autoencoder model
        if model_save_path is not None:
          mae_filename = (
              f"{type_of_cross_validation}_iteration_{i}_"
              f"{params['rl-model']}_{experiment_id}_n_days_"
              f"{n_days}.pth"
          )
          with open(os.path.join(model_dir,mae_filename), "wb+") as fd:
            torch.save(dl_model.state_dict(), fd)

      # Initialize and train the XGBoost model
      xgb_regressor = xgboost.XGBRegressor(
          **kfold_found_params, random_state=random_seeds[i]
      )
      xgb_regressor.fit(X_train, y_train.values)
      predicted_labels = xgb_regressor.predict(X_test)

      # Store predictions and true labels for later evaluation
      ordered_participant_ids.append(participant_ids_test)
      all_predictions.append(predicted_labels)
      all_true_labels.append(y_test.values)
      # We have changed the thresholds to be 1.5 instead of the previous 1.1!
      current_class_predictions = regression_to_classification(
          predicted_labels, [1.5, 2.9]
      )
      current_class_true_labels = regression_to_classification(
          y_test, [1.5, 2.9]
      )
      clf_labels_preds.append(current_class_predictions)
      clf_labels_true_labels.append(current_class_true_labels)
      # This accuracy metric does not include mean or std, just the actual
      # values!
      accuracy_dfs.append(
          classification_metrics(
              metrics.confusion_matrix(
                  current_class_true_labels, current_class_predictions
              )
          )
      )

    # We are not saving the XGBoost models to avoid confusion with previous
    # ablation studies

    # Evaluation logic (similar to your original code)
    regression_metrics = evaluate_regression_trials(
        all_predictions, all_true_labels, verbose=False
    )
    classification_results = evaluate_classification_trials(
        clf_labels_preds, clf_labels_true_labels, labels_to_name_dic, verbose=False
    )

    subjects_and_iterations_dict = {
        "n_subjects": len(training_df),
        "n_iterations": i + 1,
        "saved_model_dir": model_dir,
    }
    # Store results for this parameter combination
    results.append(
        {
            **subjects_and_iterations_dict,
            **params,
            **regression_metrics,
            **classification_results,
            **kfold_found_params,
        }
    )

  # Save the results to a CSV file
  df_results = pd.DataFrame(results)
  if results_filename is None and path_to_save is not None:
    filename = f"{params['rl-model']}_{type_of_cross_validation}_latest.pq"

  if path_to_save is not None:
    with open(os.path.join(path_to_save,filename), "wb+") as fd:
      df_results.to_parquet(fd)

  if return_results:
    return df_results, classification_results

def generate_parameter_combinations(param_grid):
  """Generates all possible combinations of parameters from a dictionary.

  Args:
      param_grid (dict): A dictionary where keys are parameter names and values
          are lists of possible values for each parameter.

  Returns:
      list: A list of dictionaries, where each dictionary represents one combination
          of parameters.
  """

  all_combinations = [{}]
  for key, values in param_grid.items():
    new_combinations = []
    for value in values:
      for combination in all_combinations:
        new_combination = combination.copy()
        new_combination[key] = value
        new_combinations.append(new_combination)
    all_combinations = new_combinations
  return all_combinations

class MaskedAutoencoder(nn.Module):
  """
  Masked autoencoder model with biomarker embeddings.
  """

  def __init__(self, input_dim, latent_space: int = 32):
    super().__init__()
    first_layer_dim= max(2*input_dim, 64)
    self.encoder = nn.Sequential(
        nn.Linear(input_dim, first_layer_dim),
        nn.LeakyReLU(),
        nn.Linear(first_layer_dim, latent_space)
    )
    self.decoder = nn.Sequential(
        nn.Linear(latent_space, input_dim), nn.LeakyReLU()
    )

  def forward(self, x, mask=None, return_encoded=False):
    if mask is None:
      mask = torch.ones_like(x)

    masked_embeddings = x * mask

    # Encode and decode
    encoded = self.encoder(masked_embeddings)
    if return_encoded:
      return encoded, self.decoder(encoded)

    return self.decoder(encoded)

class SimpleAutoencoder(nn.Module):
  """
  Masked autoencoder model with biomarker embeddings.
  """

  def __init__(self, input_dim, latent_space: int = 32):
    super().__init__()
    first_layer_dim= max(2*input_dim, 64)
    self.encoder = nn.Sequential(
        nn.Linear(input_dim, first_layer_dim),
        nn.LeakyReLU(),
        nn.Linear(first_layer_dim, latent_space)
    )
    self.decoder = nn.Sequential(
        nn.Linear(latent_space, input_dim), nn.LeakyReLU()
    )

  def forward(self, x, return_encoded=False):
    # Encode and decode
    encoded = self.encoder(x)
    if return_encoded:
      return encoded, self.decoder(encoded)

    return self.decoder(encoded)

In [ ]:
training_data_aggregated_all_days

,participant_id,RHR_FreeLiving_mean,RHR_FreeLiving_median,RHR_FreeLiving_std,HRV_FreeLiving_mean,HRV_FreeLiving_median,HRV_FreeLiving_std,STEPS_Daily_mean,STEPS_Daily_median,STEPS_Daily_std,...,framingham_risk_category,optimal_framingham_risk,framingham30_risk,framingham30_risk_category,optimal_framingham30_risk,ascvd_risk,ascvd_risk_category,optimal_ascvd_risk,apri,hypertension_binary
0,12591,65.509615,65.0,1.960948,25.452000,25.0305,4.018824,7329.278846,6557.5,3087.041901,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.350877,1
1,12597,49.201923,49.0,1.623379,15.998155,15.9660,2.052595,5357.019231,4956.0,2354.257334,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.503513,1
2,12599,65.168421,64.0,4.636302,42.677238,43.4890,10.082448,9036.231579,8172.0,4653.169485,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.214918,0
3,12612,71.080000,71.0,1.796068,44.996636,46.0220,7.762804,10998.481481,9806.0,5084.491732,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.220629,0
4,12614,65.200000,64.0,4.300286,22.555556,22.2240,5.748754,5953.837438,5469.0,3471.700982,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.612245,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,199756,68.432099,69.0,3.625096,22.274354,22.0030,6.682359,7019.839506,7004.0,2800.065485,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.171086,0
1169,199757,65.696078,65.0,3.522732,11.017429,10.9355,1.574011,11991.941176,10074.0,7029.545331,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.249653,1
1170,199758,64.527132,65.0,2.334240,28.951650,27.5440,7.033047,5076.480620,5088.0,2297.668607,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.151313,0
1171,199759,69.549451,69.0,3.029373,14.461491,14.5800,3.405676,5321.453608,4643.0,4216.633141,...,None,NaN,NaN,None,NaN,NaN,None,NaN,0.234962,0


In [ ]:
# @title KFold Cross Validation Ablation
param_grid = {
        'rl-model': ['masked_autoencoder'],
        'regression_model': ['XGBoost'],
        'learner': ['l1-l2'],
        'latent_space': [64],
        'n_days': [120],
        'experiment_name': [EXP_1, EXP_2, EXP_3, EXP_4, EXP_5, EXP_6,
                            EXP_7, EXP_8, EXP_9, EXP_10, EXP_11, EXP_12,
                            EXP_13,EXP_14, EXP_15, EXP_16, EXP_17,
                            EXP_18, EXP_19, EXP_20, EXP_21, EXP_22,
                            EXP_23, EXP_24, EXP_25, EXP_26, EXP_27, EXP_28],
        'type_of_cross_validation': ['kfold']
    }

training_details_df, classification_results = run_dl_ablation_study(param_grid,
                   path_to_save=OUTPUTS_K_FOLD_ABLATION_FOLDER,
                   training_data=training_data_aggregated_all_days,
                   results_filename=None,
                   model_save_path=OUTPUTS_MODELS_PATH,
                   return_results=True,
                   ae_type = "masked_autoencoder"
                  )

  EXP_1:   0%|          | 0/5 [00:00<?, ?it/s]

------



  EXP_2:   0%|          | 0/5 [00:00<?, ?it/s]

------



  EXP_3:   0%|          | 0/5 [00:00<?, ?it/s]

...


  EXP_27:   0%|          | 0/5 [00:00<?, ?it/s]

------



  EXP_28:   0%|          | 0/5 [00:00<?, ?it/s]

------



Ablation Study Progress: 100%|██████████| 28/28 [10:08<00:00, 21.74s/it]


In [ ]:
training_details_df

,n_subjects,n_iterations,saved_model_dir,rl-model,regression_model,learner,latent_space,n_days,experiment_name,type_of_cross_validation,...,auroc_avg,auroc_std,auprc_avg,auprc_std,booster,learning_rate,n_estimators,eta,reg_alpha,reg_lambda
0,1168,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'name': 'EXP_1', 'description': 'Fitbit + Dem...",kfold,...,0.715357,0.018482,0.628732,0.033118,gblinear,0.19,50,0.1,0,0
1,1173,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'name': 'EXP_2', 'description': 'Fitbit subse...",kfold,...,0.701678,0.018770,0.613295,0.025471,gblinear,0.19,50,0.1,0,0
2,1173,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'name': 'EXP_3', 'description': 'Fitbit subse...",kfold,...,0.642578,0.032452,0.541611,0.050222,gblinear,0.19,50,0.1,0,0
3,1173,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'name': 'EXP_4', 'description': 'Demographics...",kfold,...,0.698591,0.020240,0.610805,0.014241,gblinear,0.19,50,0.1,0,0
4,1164,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'name': 'EXP_5', 'description': 'Fitbit + Dem...",kfold,...,0.736672,0.010505,0.654682,0.021229,gblinear,0.19,50,0.1,0,0
5,1169,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'name': 'EXP_6', 'description': 'Demographics...",kfold,...,0.733130,0.020774,0.647631,0.021366,gblinear,0.19,50,0.1,0,0
6,1169,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'name': 'EXP_7', 'description': 'HbA1c', 'fea...",kfold,...,0.635605,0.030284,0.552963,0.045214,gblinear,0.19,50,0.1,0,0
7,1173,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'name': 'EXP_8', 'description': 'Demographics...",kfold,...,0.774092,0.015473,0.690737,0.011985,gblinear,0.19,50,0.1,0,0
8,1173,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'name': 'EXP_9', 'description': 'Fasting Gluc...",kfold,...,0.696530,0.034454,0.610108,0.037161,gblinear,0.19,50,0.1,0,0
9,1168,5,/output_data/autoencoder_ablation_study_result...,masked_autoencoder,XGBoost,l1-l2,64,120,"{'name': 'EXP_10', 'description': 'Fitbit + De...",kfold,...,0.770665,0.030574,0.690977,0.038437,gblinear,0.19,50,0.1,0,0


In [ ]:
# @title KFold Cross Validation Ablation
param_grid = {
        'rl-model': ['simple_autoencoder'],
        'regression_model': ['XGBoost'],
        'learner': ['l1-l2'],
        'latent_space': [64],
        'n_days': [120],
        'experiment_name': [EXP_1, EXP_2, EXP_3, EXP_4, EXP_5, EXP_6,
                            EXP_7, EXP_8, EXP_9, EXP_10, EXP_11, EXP_12,
                            EXP_13,EXP_14, EXP_15, EXP_16, EXP_17,
                            EXP_18, EXP_19, EXP_20, EXP_21, EXP_22,
                            EXP_23, EXP_24, EXP_25, EXP_26, EXP_27, EXP_28
                           ],
        'type_of_cross_validation': ['kfold']
    }

training_details_df, classification_results= run_dl_ablation_study(param_grid,
                   path_to_save=OUTPUTS_K_FOLD_ABLATION_FOLDER,
                   training_data=training_data_aggregated_all_days,
                   results_filename=None,
                   model_save_path=OUTPUTS_MODELS_PATH,
                   return_results=True,
                   ae_type = "autoencoder"
                  )

  EXP_1:   0%|          | 0/5 [00:00<?, ?it/s]

------



  EXP_2:   0%|          | 0/5 [00:00<?, ?it/s]

------



  EXP_3:   0%|          | 0/5 [00:00<?, ?it/s]

...


  EXP_27:   0%|          | 0/5 [00:00<?, ?it/s]

------



  EXP_28:   0%|          | 0/5 [00:00<?, ?it/s]

------



Ablation Study Progress: 100%|██████████| 28/28 [09:10<00:00, 19.67s/it]


In [ ]:
training_details_df

# Download files

Expected files to be downloaded

* `output_data/autoencoder_ablation_study_results/masked_autoencoder_kfold_latest.pq`
* `output_data/autoencoder_ablation_study_results/simple_autoencoder_kfold_latest.pq`
* `output_data/autoencoder_ablation_study_results.zip`

In [ ]:
files.download(os.path.join(OUTPUTS_K_FOLD_ABLATION_FOLDER, 'masked_autoencoder_kfold_latest.pq'))
files.download(os.path.join(OUTPUTS_K_FOLD_ABLATION_FOLDER, 'simple_autoencoder_kfold_latest.pq'))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
zip_filename = f'{EXPERIMENT_NAME}.zip'
ZIP_PATH = os.path.join(OUTPUTS_FOLDER, zip_filename)

In [ ]:
!zip -r $ZIP_PATH $OUTPUTS_K_FOLD_ABLATION_FOLDER

  adding: output_data/autoencoder_ablation_study_results/ (stored 0%)
  adding: output_data/autoencoder_ablation_study_results/simple_autoencoder_kfold_latest.pq (deflated 65%)
  adding: output_data/autoencoder_ablation_study_results/masked_autoencoder_kfold_latest.pq (deflated 65%)
  adding: output_data/autoencoder_ablation_study_results/ablation_models/ (stored 0%)
  adding: output_data/autoencoder_ablation_study_results/ablation_models/simple_autoencoder_EXP_2_kfold_n_days_120/ (stored 0%)
  adding: output_data/autoencoder_ablation_study_results/ablation_models/simple_autoencoder_EXP_2_kfold_n_days_120/kfold_iteration_4_simple_autoencoder_EXP_2_n_days_120.pth (deflated 12%)
  adding: output_data/autoencoder_ablation_study_results/ablation_models/simple_autoencoder_EXP_2_kfold_n_days_120/kfold_iteration_3_simple_autoencoder_EXP_2_n_days_120.pth (deflated 12%)
  adding: output_data/autoencoder_ablation_study_results/ablation_models/simple_autoencoder_EXP_2_kfold_n_days_120/kfold_itera

In [ ]:
files.download(ZIP_PATH)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>